In [1]:
from srgan import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2904604870509937389,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6115084048863163887
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5331917476420946811
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5498994688
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9929674567164418076
 physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [4]:
vgg_path = 'vgg16_notop.hdf5'
# input_shape=(270, 480, 3)
# tgt_shape=(1080, 1920, 3)

# input_shape=(180, 320, 3)
# tgt_shape=(720, 1280, 3)

input_shape=(90, 160, 3)
tgt_shape=(360, 640, 3)

# input_shape=(60, 106, 3)
# tgt_shape=(240, 424, 3)

batch_size = 8
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
srgan.vgg_combined.get_weights()[0]

array([[[[-0.00185297, -0.07379565, -0.01964342, ..., -0.01883738,
          -0.06054966,  0.04660441],
         [ 0.05767332, -0.05687572,  0.01513757, ..., -0.08152913,
           0.0960612 ,  0.0506926 ],
         [ 0.04582876,  0.01280163, -0.06316902, ..., -0.00549409,
           0.06770267, -0.08546918]],

        [[ 0.03858781, -0.08359809, -0.09724455, ...,  0.08145809,
           0.07477872,  0.08851822],
         [-0.05129045, -0.07133273,  0.04290502, ...,  0.07680146,
           0.04910679, -0.01044362],
         [-0.07422568, -0.05944311,  0.08665128, ..., -0.00848309,
           0.05267707, -0.0008912 ]],

        [[ 0.08782899,  0.00200879,  0.02319129, ..., -0.03644748,
          -0.07488562,  0.01721133],
         [ 0.0462224 ,  0.02894387, -0.01261906, ...,  0.0399228 ,
          -0.04153217, -0.03049976],
         [ 0.06741188,  0.08457746,  0.05463663, ..., -0.01752547,
           0.0350382 , -0.01999172]]],


       [[[-0.09422287,  0.03519845, -0.04212252, ..., -0

In [6]:
input_dir = 'data/low/'
tgt_dir = 'data/high/'
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)

In [7]:
for e in range(10):
    print('epoch', e)
    data = gen.flow_from_directory().__next__()
    X_low = data[0]
    X_high = data[1]
    srgan.train(X_low, X_high)

epoch 0
d_loss 0.9000134468078613
epoch 1
d_loss 0.8859623670578003
epoch 2
d_loss 0.873175323009491
epoch 3
d_loss 0.8502940535545349
epoch 4
d_loss 0.8256518840789795
epoch 5
d_loss 0.8116995692253113
epoch 6
d_loss 0.7786446809768677
epoch 7
d_loss 0.7342914342880249
epoch 8
d_loss 0.677404522895813
epoch 9
d_loss 0.6283054947853088


In [8]:
srgan.vgg_combined.get_weights()[0]

array([[[[-0.00401447, -0.07293294, -0.02175235, ..., -0.02061155,
          -0.06318936,  0.04545167],
         [ 0.05527773, -0.05663117,  0.01586076, ..., -0.08306351,
           0.09357951,  0.04822547],
         [ 0.04351152,  0.01317796, -0.06256503, ..., -0.00708133,
           0.06480643, -0.08817061]],

        [[ 0.0386387 , -0.08638565, -0.09422427, ...,  0.07744472,
           0.07193625,  0.09149268],
         [-0.05131909, -0.07447121,  0.0473152 , ...,  0.07285726,
           0.04637393, -0.00817256],
         [-0.07444035, -0.06258551,  0.09096412, ..., -0.01250353,
           0.04972558,  0.00119107]],

        [[ 0.08963544,  0.00338504,  0.02650165, ..., -0.03467352,
          -0.07557421,  0.0136501 ],
         [ 0.04819985,  0.02874333, -0.00800937, ...,  0.0417686 ,
          -0.04118557, -0.03423614],
         [ 0.06917934,  0.08451273,  0.059147  , ..., -0.01576075,
           0.03448588, -0.02375792]]],


       [[[-0.09679991,  0.03747332, -0.0450968 , ..., -0